# state probabilities

imports

In [ ]:
import json

import fsspec
import xarray as xr

from pangeo_fish.hmm.estimator import EagerScoreEstimator
from pangeo_fish.pdf import combine_emission_pdf

## parametrize with [papermill](https://papermill.readthedocs.io/en/latest/)

In [ ]:
# Notebook specification
cluster_size: int = 1

# Dask parameters (Machine, and configuration dependent)
# cluster_size: int | None = None
cluster_name: str = "datarmor-local"
cluster_overrides: dict = {}

# Run specific
working_path: str = "/home/datawork-taos-s/public/fish/"
tag_name: str = "A18832"
tag_base_path: str = "/home/datawork-lops-iaocea/data/fish-intel/tag/nc/"
tag_db_path: str = (
    "/home/datawork-lops-iaocea/data/fish-intel/acoustic/FishIntel_tagging_France.csv"
)
detections_path: str = "/home/datawork-lops-iaocea/data/fish-intel/acoustic/detections_recaptured_fishintel.csv"
ref_model_name: str = "copernicus"
nside: int = 4096  # healpix resolution

# we can set the parameter acoustic to use acoustic information or not
acoustic: str = "/acoustic"

## set path using the parameters


In [ ]:
tag_url = tag_base_path + tag_name + ".nc"
input_path = (
    working_path
    + tag_name
    + "/"
    + ref_model_name
    + acoustic
    + "/emission_"
    + str(nside)
    + ".zarr"
)
parameter_path = (
    working_path
    + tag_name
    + "/"
    + ref_model_name
    + acoustic
    + "/sigma_"
    + str(nside)
    + ".json"
)
output_path = (
    working_path
    + tag_name
    + "/"
    + ref_model_name
    + acoustic
    + "/state_"
    + str(nside)
    + ".zarr"
)

## Start Dask cluster

In [ ]:
import dask_hpcconfig
from distributed import Client

In [ ]:
domainname = !domainname

if domainname == ["nisdatarmor"]:
    cluster = dask_hpcconfig.cluster(cluster_name, **cluster_overrides)
    if cluster_size is not None:
        cluster.scale(cluster_size)
else:
    cluster = dask_hpcconfig.cluster("local")

client = Client(cluster)
client

open the emission probabilities

In [ ]:
emission = (
    xr.open_dataset(input_path, engine="zarr", chunks={}, inline_array=True)
    .pipe(combine_emission_pdf)
    .drop_vars("resolution")
)
emission

read the parameters

In [ ]:
with fsspec.open(parameter_path, mode="r") as f:
    parameters = json.load(f)
parameters.pop("tolerance", None)
parameters

create the estimator

In [ ]:
%%time
estimator = EagerScoreEstimator(**parameters)
estimator

compute the state probabilities

In [ ]:
%%time
states = estimator.predict_proba(emission)
states

write to disk

In [ ]:
%%time
states.chunk({"time": "auto", "x": -1, "y": -1}).to_zarr(
    output_path, mode="w", consolidated=True
)

## visualization

In [ ]:
states_ = xr.open_dataset(output_path, engine="zarr", chunks={})
states_

In [ ]:
states_.states.count(dim=["x", "y"]).plot()

In [ ]:
states_["states"].isel(time=5).plot(x="longitude", y="latitude")